In [ ]:
import pandas as pd
import os

# 1️⃣ Folder containing all your Excel files
folder_path = r"C:\Users\HP\OneDrive\Desktop\Mophones case study\Credit Data\Credit Data"  # <-- change this

# 2️⃣ List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.csv')]
print("Files found:", csv_files)

# 3️⃣ Read all CSVs into a list of DataFrames
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 4️⃣ Merge all DataFrames, automatically aligning columns
merged_df = pd.concat(dfs, ignore_index=True, sort=False)

# 5️⃣ Fill missing values with "unknown"
merged_df = merged_df.fillna("unknown")

# Optional: check the merged data
print(merged_df.head())
print("Columns in merged file:", merged_df.columns)


# 6️⃣ Save merged data to Downloads folder
downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")
merged_file_path = os.path.join(downloads_path, "mophones_merged.csv")

merged_df.to_csv(r"C:\Users\HP\OneDrive\Desktop\Mophones case study02.csv", index=False)

print("Merged CSV created successfully in Downloads:", merged_file_path)




Files found: ['Credit Data - 01-01-2025.csv', 'Credit Data - 30-03-2025.csv', 'Credit Data - 30-06-2025.csv', 'Credit Data - 30-09-2025.csv', 'Credit Data - 30-12-2025.csv']
             LOAN_ID      DATE  CUSTOMER_AGE  TOTAL_PAID TOTAL_DUE_TODAY  \
0  recnBZuKLsWCIm3cW  1/1/2025           102       20639         21649.0   
1  recwUTOdXSW5kSAed  1/1/2025           211       48101         49419.0   
2  recPCzNo2d9qK1VB3  1/1/2025            22       52999         52999.0   
3  rec98Vut24ajzNeWL  1/1/2025           145       21899         48249.0   
4  recEmhLuKhmOsEqGl  1/1/2025           271       35719         35419.0   

   BALANCE  DAYS_PAST_DUE CLOSING_BALANCE  ADVANCE BALANCE_DUE_TO_DATE  ...  \
0  38380.0              4         38380.0      0.0             -1010.0  ...   
1  29038.0              0             0.0      0.0             -1318.0  ...   
2      0.0              0             0.0      0.0                 0.0  ...   
3  80600.0            110         80600.0      0.0   

In [3]:
import pandas as pd
import numpy as np

# 1️⃣ Read all sheets
import pandas as pd

file_path = r"C:\Users\HP\OneDrive\Desktop\Mophones case study\Sales and Customer Data.xlsx"

df = pd.read_excel(file_path, sheet_name="Sales Details")
print(df.shape)


(1048575, 16)


In [ ]:
#Standardize column names
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

print(df.columns.tolist())


['sale_id', 'sale_date', 'returned', 'return_date', 'sale_type', 'seller', 'seller_type', 'return_policy_compliance', 'cash_price', 'loan_price', 'client_model', 'business_model', 'loan_term', 'product_name', 'model', 'loan_id']


In [5]:
if "loan_id" not in df.columns:
    raise ValueError("loan_id column still not found after cleaning")


In [6]:
date_cols = ["sale_date", "return_date"]

for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")


In [7]:
# Returned flag
df["returned"] = df["returned"].fillna(0).astype(int)


In [8]:
# Price columns
price_cols = ["cash_price", "loan_price"]

for col in price_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")


In [9]:
df[["sale_id", "loan_id"]].head()
df.drop(columns=["sale_id"], inplace=True)



In [10]:
categorical_cols = [
    "seller_type",
    "return_policy_compliance",
    "product_model"
]

for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].fillna("Unknown")


In [11]:
df["loan_premium_pct"] = (
    (df["loan_price"] - df["cash_price"]) / df["cash_price"]
) * 100


In [12]:
df["days_to_return"] = (
    df["return_date"] - df["sale_date"]
).dt.days


In [13]:
df["sale_month"] = df["sale_date"].dt.to_period("M").astype(str)
df["sale_year"] = df["sale_date"].dt.year


In [14]:
gender_df = pd.read_excel(file_path, sheet_name="Gender")
gender_df.columns = gender_df.columns.str.strip().str.lower().str.replace(" ", "_")


In [17]:
dob_df = pd.read_excel(file_path, sheet_name="DOB")
dob_df.columns = dob_df.columns.str.strip().str.lower().str.replace(" ", "_")
dob_df.rename(columns={"date_of_birth": "dob"}, inplace=True)



In [16]:
print(dob_df.columns.tolist())


['_id', 'provider', 'date_of_birth', 'loan_id', 'createdat_utc']


In [18]:
income_df = pd.read_excel(file_path, sheet_name="Income Level")
income_df.columns = income_df.columns.str.strip().str.lower().str.replace(" ", "_")


In [19]:
df = df.merge(gender_df, on="loan_id", how="left")
df = df.merge(dob_df, on="loan_id", how="left")
df = df.merge(income_df, on="loan_id", how="left")


MemoryError: Unable to allocate 7.73 TiB for an array with shape (1062496961937,) and data type int64

In [24]:
# Sort by creation date (most recent first)
gender_df = gender_df.sort_values("createdat_utc", ascending=True)

# Keep only one row per loan_id (last one = most recent)
gender_df = gender_df.drop_duplicates(subset="loan_id", keep="last")

# Keep only relevant columns
gender_df = gender_df[["loan_id", "gender"]]

# Make sure loan_id is unique
assert gender_df["loan_id"].is_unique


KeyError: 'createdat_utc'

In [ ]:

gender_df = (
    gender_df
    .sort_values("createdat_utc")
    .drop_duplicates(subset="loan_id", keep="last")
)

assert gender_df["loan_id"].is_unique

gender_df = gender_df[["loan_id", "gender"]]

print(gender_df.columns.tolist())

gender_df["gender"] = gender_df["gender"].astype("category")
df["loan_id"] = df["loan_id"].astype("string")
gender_df["loan_id"] = gender_df["loan_id"].astype("string")

df = df.merge(
    gender_df,
    on="loan_id",
    how="left",
    validate="many_to_one"  # 👈 CRITICAL SAFETY CHECK
)
dob_df = (
    dob_df
    .sort_values("createdat_utc")
    .drop_duplicates("loan_id", keep="last")
)[["loan_id", "dob"]]

assert dob_df["loan_id"].is_unique

income_df = (
    income_df
    .sort_values("createdat_utc")
    .drop_duplicates("loan_id", keep="last")
)[["loan_id", "income_level"]]

df = df.merge(gender_df, on="loan_id", how="left", validate="many_to_one")
df = df.merge(dob_df, on="loan_id", how="left", validate="many_to_one")
df = df.merge(income_df, on="loan_id", how="left", validate="many_to_one")





KeyError: 'createdat_utc'

In [23]:
print(gender_df.columns.tolist())



['loan_id', 'citizenship', 'gender']


### NPS DATA CLEANING 

In [27]:
import pandas as pd

pf = pd.read_excel(r"C:\Users\HP\OneDrive\Desktop\Mophones case study\NPS Data (1).xlsx")

pf = pf.rename(columns={
    "Submission ID": "submission_id",
    "Respondent ID": "respondent_id",
    "Submitted at": "submitted_at",
    "Loan Id": "loan_id",
    "Using a scale from 0 (not likely) to 10 (very likely), how likely are you to recommend MoPhones to friends or family?": "nps_score",
    "What is the main reason for your score?": "nps_reason",
    "What is one thing we could do to improve your experience with us?": "improvement_suggestion",
    "Are you happy with the quality and performance of your MoPhones device?": "device_quality_happy",
    "Are you happy with the service and support provided by MoPhones?": "service_support_happy",
    "Have you ever experienced a delay in your payment reflecting in your Mophones account?": "payment_delay",
    "Have you ever had difficulty getting assistance from MoPhones customer support when needed?": "support_difficulty",
    "(If Yes) – Please describe the challenge you faced and how we can improve your experience.": "support_challenge_details",
    "Have you experienced any battery-related issues with your MoPhones device?": "battery_issues",
    "Have you used the MoPhones app (MoApp) to manage your account or make payments?": "used_moapp",
    "Which communication channel do you prefer when contacting MoPhones for inquiries or support?": "preferred_channel",
    "Have you ever had your phone lock despite making a payment on time?": "phone_locked_on_time_payment",
    "Any other Feedback?": "other_feedback"
})


In [28]:
# Ensure datetime
pf["submitted_at"] = pd.to_datetime(pf["submitted_at"])

# Ensure NPS score is numeric
pf["nps_score"] = pd.to_numeric(pf["nps_score"], errors="coerce")


In [29]:
def nps_category(score):
    if pd.isna(score):
        return None
    elif score >= 9:
        return "Promoter"
    elif score >= 7:
        return "Passive"
    else:
        return "Detractor"

df["nps_category"] = df["nps_score"].apply(nps_category)


In [30]:
nps_summary = (
    df["nps_category"]
    .value_counts(normalize=True) * 100
)

overall_nps = nps_summary.get("Promoter", 0) - nps_summary.get("Detractor", 0)

overall_nps


5.922208281053955

In [31]:
yes_no_cols = [
    "device_quality_happy",
    "service_support_happy",
    "payment_delay",
    "support_difficulty",
    "battery_issues",
    "used_moapp",
    "phone_locked_on_time_payment"
]

for col in yes_no_cols:
    df[col] = (
        df[col]
        .str.strip()
        .str.lower()
        .replace({"yes": "Yes", "no": "No"})
    )


In [32]:
df["year"] = df["submitted_at"].dt.year
df["month"] = df["submitted_at"].dt.to_period("M")
